# Sagemaker deployment

In [5]:
import tensorflow
tensorflow.__version__

'2.4.1'

In [6]:
import json

import sagemaker
from sagemaker.tensorflow import TensorFlow
from sagemaker import get_execution_role

sess = sagemaker.Session()

role = "arn:aws:iam::908177370303:role/demoexample-default-sagemaker-role" #get_execution_role()

output_path='s3://' + sess.default_bucket() + '/tensorflow/tweet-sentiments'
prefix = 'tensorflow/tweets'
bucket = sess.default_bucket()

In [47]:
!python train.py --epochs 2 --train data4 --test data4 --model-dir model

2021-03-21 21:48:01.404398: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-03-21 21:48:01.404594: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-03-21 21:48:01.520245: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
Epoch 1/2
462/462 [==============================] - 24s 50ms/step - loss: 0.6893 - accuracy: 0.5382 - val_loss: 0.6472 - val_accuracy: 0.7354
Epoch 2/2
462/462 [==============================] - 23s 50ms/step - loss: 0.6021 - accuracy: 0.7380 - val_loss: 0.5364 - val_accuracy: 0.7847
2021-03-21 21:48:51.339537: W tensorflow/python/util/util.cc:348] Sets are not c

In [56]:
local_mode=False

if local_mode:
    instance_type='local'
else:
    instance_type='ml.c4.xlarge'
    
est = TensorFlow(
    entry_point='train.py',
    source_dir='.', # directory of your training script
    role=role,
    framework_version='2.3.1',
    model_dir=False, # don't pass --model_dir to your training script
    py_version='py37',
    instance_type=instance_type,
    instance_count=1,
    output_path=output_path,
    hyperparameters={
        'epochs':1,
        'learning-rate': 1e-4
    }
)

In [ ]:

loc = sess.upload_data(path='data4', bucket=bucket, key_prefix=prefix)
channels = {
    "training": loc,
    "testing": loc
}

In [ ]:
instance_type

In [57]:
est.fit(inputs=channels)


2021-03-21 14:04:05 Starting - Starting the training job...
2021-03-21 14:04:08 Starting - Launching requested ML instancesProfilerReport-1616335442: InProgress
......
2021-03-21 14:05:35 Starting - Preparing the instances for training.........
2021-03-21 14:06:56 Downloading - Downloading input data...
2021-03-21 14:07:36 Training - Downloading the training image2021-03-21 14:07:25.310843: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2021-03-21 14:07:25.320774: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2021-03-21 14:07:25.689360: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2021-03-21 14:07:29,540 sagemaker-training-toolkit INFO     Imported framework sagemaker_tensorflow_container.training
2021-03-21 14:07:29,546 sagemaker-training

In [8]:
from sagemaker.tensorflow import TensorFlowModel

training_job = "tensorflow-training-2021-03-21-14-04-02-356"
model = TensorFlowModel(
    role=role,
    source_dir=".",
    entry_point='inference.py',
    model_data=f"s3://{bucket}/tensorflow/tweet-sentiments/{training_job}/output/model.tar.gz",
    framework_version='2.3.1',
    py_version='py3'
)

In [9]:
# set local_mode to False if you want to deploy on a remote
# SageMaker instance

local_mode=False

if local_mode:
    instance_type='local'
else:
    instance_type='ml.c4.xlarge'

predictor = model.deploy(
    endpoint_name="tweet-sentiments",
    initial_instance_count=1,
    instance_type=instance_type,
    )

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


-------------!

In [10]:
predictor.predict("What better way to relax over a nice meal and a bottle of your favorite wine?")

{'predictions': [[0.865202308]]}

In [ ]:
!pip install tensorflow